In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import dateutil.tz

import tensorflow as tf

# Functions

## Create a dataframe for each type of activity by connecting smaller dataframes

### Implement the necessary columns in smaller dataframes

In [ ]:
def add_datetime_column(df, timestamp_col_name, local_timezone=datetime.timezone.utc, reordering=False):
    """ A function to add a 'datetime' column that contains information about the date and time of the recording
    Args:
        1) df - a dataframe that contains the required data
        2) timestamp_col_name - the name of the df dataframe column that contains the timestamp information
        3) local_timezone - the time zone in which the measurements were made (object of class 'dateutil.tz.tz.tzlocal')
        4) reordering - rearrange the dataframe columns so that the 'datetime' column is next to the 'timestamp' column
    Returns:
        A dataframe that contains a 'datetime' column
    """
    df['datetime'] = df[timestamp_col_name].apply(lambda timestamp:
        datetime.datetime.fromtimestamp(timestamp / 1000.0, tz=local_timezone).strftime("%d.%m.%Y, %H:%M:%S.%f")[:-3])
    df['datetime'] = df['datetime'].apply(lambda timestamp_str: 
        datetime.datetime.strptime(timestamp_str, "%d.%m.%Y, %H:%M:%S.%f"))
    
    if reordering:
        # Reorder the columns
        new_column_order = [timestamp_col_name, 'datetime'] + [col for col in df.columns if col != timestamp_col_name and col != 'datetime']
        df = df.reindex(columns=new_column_order)
    return df


def add_time_column(df, timestamp_col_name, initial_time=0, timecol_position=1):
    """A function to add a 'time' column to the original dataframe, which contains the recording time relative to the initial time instant_time
    Args:
        1) df - a dataframe that contains the required data
        2) timestamp_col_name - the name of the df dataframe column that contains the timestamp information
        3) initial_time - initial moment of time (s)
        4) timecol_position - index of column 'time'position among other columns (column indexing starts at 0)
    Returns:
        A dataframe that contains a 'time' column
    """
    # Convert the 'timestamp' column to pandas Timestamp type
    df['temp_timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Calculate the time difference relative to the start
    df['time'] = initial_time + (df['temp_timestamp'] - df['temp_timestamp'].iloc[0]).dt.total_seconds()
    
    # Drop unnecessary 'temp_timestamp' column
    df.drop('temp_timestamp', axis=1, inplace=True)
    
    # Pop the 'time' column and store it in a variable
    time_column = df.pop('time')

    # Insert the 'time' column at the desired position
    df.insert(timecol_position, 'time', time_column)
    return df

### Concatenate smaller dataframes to produce a single dataframe

In [ ]:
def concat_dataframes(dfs, timestamp_col_name, timecol_position=1):
    """ A function to concatenate dataframes that contain data about measurements taken at different times (eg, different hours, days, months) and a timestamp column, so that the 'time' column of the output dataframe displays the time as if the measurements were occurring continuously
    Args:
        1) dfs - the list of dataframes, that contain timestamp column, and from which the output dataframe will consist
        2) timestamp_col_name - the name of the df dataframe column that contains the timestamp information
        3) timecol_position - index of column 'time'position among other columns (column indexing starts at 0) 
    Returns:
        A dataframe concatenated_df, whose "time" column shows the time as if the measurements were continuous
    """
    dfs[0] = add_time_column(df=dfs[0], timestamp_col_name=timestamp_col_name, initial_time=0, timecol_position=timecol_position)
    for i in range(1, len(dfs)):
        time_diffs = dfs[(i-1)][timestamp_col_name].diff()
        init_time = dfs[(i-1)]['time'].iloc[-1] + round(time_diffs.mean() / 1000, 3)
        dfs[i] = add_time_column(df=dfs[i], timestamp_col_name=timestamp_col_name, initial_time=init_time, timecol_position=timecol_position)
        
    # Concatenate dfs
    concatenated_df = pd.concat(dfs)
    concatenated_df.reset_index(inplace=True)
    concatenated_df.drop('index', axis=1, inplace=True)
    
    return concatenated_df

## Create activity dataframe

In [ ]:
def create_activities_df(path):
    """ A function for creating a dataframe containing information about the beginning, end and name of each physical activity
    Args:
        1) path - the path to the Label Studio file that contains the labeled signals
    Returns:
        Dataframe containing information about the beginning, end and name of each physical activity
    """
    labeled_activities = pd.read_csv(path)
    labels = labeled_activities['label']
    labels = np.array(labels)[0]    # the original type of labels is pd.Series
    labels = labels.split('},')
    
    activities_dict = {'start_time': [], 'end_time': [], 'activity': []}
    for label in labels:
        start_time, end_time, _, activity = label.split(',')
        activities_dict['start_time'].append(float(start_time.split(':')[1]))
        activities_dict['end_time'].append(float(end_time.split(':')[1]))
        activities_dict['activity'].append(activity.split(':')[1].strip('[]}"'))
    
    activities_df = pd.DataFrame.from_dict(activities_dict)
    activities_df = activities_df.sort_values(by='start_time').reset_index(drop=True)
    return activities_df

def add_activity_column(path, df, df_merge_col):
    """ Function to add 'activity' column to dataframe df
    Args:
        1) path - the path to the Label Studio file that contains the labeled signals
        2) df - a dataframe that contains the results of accelerometer and gyroscope measurements
        3) df_merge_col - the name of the column of the dataframe df, which contains data about the measurement time ('time' column)
    """
    activities_df_local = create_activities_df(path)
    merged_df = pd.merge_asof(df, activities_df_local, left_on=df_merge_col, right_on='start_time', direction='backward')
    df['activity'] = merged_df['activity']
    
    
def display_activity_freq_spectrum_one_axes(df, activity_name, axes_name, sampling_rate, is_divided_by_g=False, x_lim=10, color='blue'):
    """ Function to display the frequency spectrum of the specified type of activity on the OX, OY and OZ axes for the specified device 
    (accelerometer or gyroscope)
    Args:
        1) df - a dataframe that contains the results of accelerometer and gyroscope measurements
        2) activity_name - selected activity type (among the values of the 'activity' column of df)
        3) axes_name - the column name of the dataframe df that contain the measurement results for the OX, OY, or OZ axes
        4) sampling_rate - actual sampling rate of your dataset (samples per second)
        5) is_divided_by_g - True if the content of the column was divided by the free fall acceleration g (g=9.81 m/s^2)
        6) x_lim - the limits of the graph along the OX axis are [-x_lim; x_lim]
        7) color - the color of the graph
    """
    activity_data = df[df['activity'] == activity_name]
    
    signal = np.array(activity_data[axes_name])
    
    if is_divided_by_g:
        signal *= 9.81
        
    fft_result = fft(signal)

    fft_freqs = fftfreq(len(activity_data), 1/sampling_rate)

    fig = plt.figure(figsize=(12, 5))
    axes = fig.add_axes([0.1, 0.1, 1, 1])
    axes.plot(fft_freqs, np.abs(fft_result), color=color)
    axes.set_title(f'{activity_name} {axes_name}')

    axes.set_xlabel('Frequency (Hz)')
    axes.set_xlim(-x_lim, x_lim)
    axes.set_ylabel('Amplitude')
    axes.grid(alpha=0.5)

    plt.show()

    
def display_activity_freq_spectrum(df, activity_name, axes_names, sampling_rate, is_divided_by_g=False, x_lim=10):
    """ Function to display the frequency spectrum of the specified type of activity on the OX, OY and OZ axes for the specified device 
    (accelerometer or gyroscope)
    Args:
        1) df - a dataframe that contains the results of accelerometer and gyroscope measurements
        2) activity_name - selected activity type (among the values of the 'activity' column of df)
        3) axes_names - an array containing the column names of the dataframe df that contain the measurement results for the OX, OY, and OZ axes, 
        respectively
        4) sampling_rate - actual sampling rate of your dataset (samples per second)
        5) is_divided_by_g - True if the content of the column was divided by the free fall acceleration g (g=9.81 m/s^2)
        6) x_lim - the limits of the graph along the OX axis are [-x_lim; x_lim]
    """
    activity_data = df[df['activity'] == activity_name]
    
    signal_x = np.array(activity_data[axes_names[0]])
    signal_y = np.array(activity_data[axes_names[1]])
    signal_z = np.array(activity_data[axes_names[2]])
    
    if is_divided_by_g:
        signal_x *= 9.81
        signal_y *= 9.81
        signal_z *= 9.81
        
    fft_x = fft(signal_x)
    fft_y = fft(signal_y)
    fft_z = fft(signal_z)

    fft_freqs = fftfreq(len(activity_data), 1/sampling_rate)

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes[0].plot(fft_freqs, np.abs(fft_x), label='X', color='blue')
    axes[0].set_title(f'{activity_name} {axes_names[0]}')

    axes[1].plot(fft_freqs, np.abs(fft_y), label='Y', color='yellow')
    axes[1].set_title(f'{activity_name} {axes_names[1]}')

    axes[2].plot(fft_freqs, np.abs(fft_z), label='Z', color='green')
    axes[2].set_title(f'{activity_name} {axes_names[2]}')

    for ax in axes:
        ax.set_xlabel('Frequency (Hz)')
        ax.set_xlim(-x_lim, x_lim)
        ax.set_xticks([i for i in range(-x_lim, x_lim+1, 2)])
        ax.set_ylabel('Amplitude')
        ax.grid(alpha=0.5)
        ax.legend()

    plt.tight_layout()
    plt.show()

## Exploring measurement time and frequency stability

In [ ]:
def get_measurement_time_df(df):
    """ A function for building a dataframe that contains information about the time of measurement of each reading of the device
    Args:
        1) df - a dataframe that contains a 'time' column
    Returns:
        time_measurement_df - dataframe that contains information about the time of measurement of each reading of the device
    """
    period_dict = {'start_time': [], 'end_time': [], 'measurement_time': []}
    prev_time = None
    curr_time = None
    for index, row in df.iterrows():
        if(index == len(df)):
            break
            
        prev_time = curr_time
        curr_time = row['time']

        if prev_time != None:
            period_dict['start_time'].append(prev_time)
            period_dict['end_time'].append(curr_time)
            period_dict['measurement_time'].append(curr_time - prev_time)
    
    time_measurement_df = pd.DataFrame.from_dict(period_dict)
    return time_measurement_df


def display_time_distribution(df, x):
    """ A function that is designed to display the measurement time distribution using a histogram
    Args:
        1) df - a dataframe that contains a column with measurement time data
        2) x - name of measurement time column
    """
    # Plot the measurement time distribution using seaborn's histogram
    # plt.figure(figsize=(15, 6)) - original size
    plt.figure(figsize=(25, 6))
    sns.histplot(df[x], bins=60, kde=True, color='blue')
    plt.xlabel('Period, s')
    plt.ylabel('Count')
    plt.title('Measurement Time Distribution')
    plt.xticks([i/1000 for i in range(0, 38, 1)])

    plt.show()


def display_data_collection_stability(df):
    """A function that displays the stability of data collection relative to the time when the measurements were taken and 
       relative to the measurement number
    Args:
        1) df - time_measurement_df
    """
    fig, axes = plt.subplots(2, 1, figsize=(15, 14))
    axes[0].plot(range(len(df[df['lost_data'] == 0])), df[df['lost_data'] == 0]['measurement_time'])
    axes[0].set_title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    axes[0].set_xlabel('number of measurement')
    axes[0].set_ylabel('measurement time')
    
    axes[1].plot(df[df['lost_data'] == 0]['start_time'], df[df['lost_data'] == 0]['measurement_time'])
    axes[1].set_title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    axes[1].set_xlabel('time of measurement')
    axes[1].set_ylabel('measurement time')
    
    plt.show()

    
def display_data_collection_stability_measurement_number(df):
    """A function that displays the stability of data collection relative to the measurement number
    Args:
        1) df - time_measurement_df
    """
    plt.figure(figsize=(12, 3))
    plt.plot(range(len(df[df['lost_data'] == 0])), df[df['lost_data'] == 0]['measurement_time'])
    plt.title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    plt.xlabel('number of measurement')
    plt.ylabel('measurement time, s')
    
    plt.show()
    

def display_data_collection_stability_measurement_time(df):
    """A function that displays the stability of data collection relative to the time when the measurements were taken
    Args:
        1) df - time_measurement_df
    """
    plt.figure(figsize=(12, 3))
    plt.plot(time_measurement_df[time_measurement_df['lost_data'] == 0]['start_time'], 
             time_measurement_df[time_measurement_df['lost_data'] == 0]['measurement_time'])
    plt.title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    plt.xlabel('time of measurement')
    plt.ylabel('measurement time')
    
    plt.show()
    

def display_data_collection_losses_measurement_number(df):
    """A function that displays the loses of data collection relative to the measurement number
    Args:
        1) df - time_measurement_df
    """
    plt.figure(figsize=(12, 3))
    plt.plot(range(len(df[df['lost_data'] == 1])), df[df['lost_data'] == 1]['measurement_time'])
    plt.title(f'Data Collection Losses (Filtered)\nAverage frequency: {freq:.3f} Hz')
    plt.xlabel('number of measurement')
    plt.ylabel('measurement time, s')
    
    plt.show()
    

def display_data_collection_losses_measurement_time(df):
    """A function that displays data collection losses relative to the time when the measurements were taken
    Args:
        1) df - time_measurement_df
    """
    plt.figure(figsize=(12, 3))
    plt.plot(time_measurement_df[time_measurement_df['lost_data'] == 1]['start_time'], 
             time_measurement_df[time_measurement_df['lost_data'] == 1]['measurement_time'])
    plt.title(f'Data Collection Losses (Filtered)\nAverage frequency: {freq:.3f} Hz')
    plt.xlabel('time of measurement, s')
    plt.ylabel('measurement time, s')
    
    plt.show()

## Visualizing results

In [ ]:
def display_three_axes(df, y, x=None, title=None, x_label=None, y_label=None, filename=None):
    """ Function for visualization of three axes (X, Y, Z) of the measurement results of the device
    Args:
        1) df - a dataframe containing the results of the device measurement
        2) x - the name of the column of the dataframe that contains the data for the OX axis (for the three lines, this is the same data)
        3) y - the list containing the dataframe column names corresponding to the OX, OY, and OZ axis measurements, respectively
        4) title - title of the graph
        5) x_label - the name of the OX axis of the graph
        6) y_label - the name of the OY axis of the graph
        7) filename - the relative path where the file will be saved (with the file name, the file extension is not required) or just the filename
    Returns:
        Nothing, but plots graph
    """
    plt.figure(figsize=(12, 6))
    if x is None:
        plt.plot(df[y[0]], label='X')
        plt.plot(df[y[1]], label='Y')
        plt.plot(df[y[2]], label='Z')
    else:
        plt.plot(df[x], df[y[0]], label='X')
        plt.plot(df[x], df[y[1]], label='Y')
        plt.plot(df[x], df[y[2]], label='Z')
        
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    if filename:
        plt.savefig(f'{filename}.png', bbox_inches='tight')
    plt.show()
    

def display_three_axes_sep(df, x, y, title=None, x_label=None, y_labels=None):
    """ Function for visualization of three axes (X, Y, Z) of the measurement results of the device, a separate graph for each axis
    Args:
        1) df - a dataframe containing the results of the device measurement;
        2) x - the name of the column of the dataframe that contains the data for the OX axis (for the three lines, this is the same data);
        3) y - the list containing the dataframe column names corresponding to the OX, OY, and OZ axis measurements, respectively;
        4) title - title of the graph;
        5) x_label - the name of the OX axis of the graph;
        6) y_labels - the list of names of the OY axis of the graph.
    Returns:
        Nothing, but plots graph
    """
    fig, axes = plt.subplots(3, 1, figsize=(10, 18))
    axes[0].plot(df[x], df[y[0]], color='blue')
    axes[0].set_xlabel(x_label)
    axes[0].set_ylabel(y_labels[0])
    axes[0].set_title(title + ' accX')

    axes[1].plot(df[x], df[y[1]], color='yellow')
    axes[1].set_xlabel(x_label)
    axes[1].set_ylabel(y_labels[1])
    axes[1].set_title(title + ' accY')

    axes[2].plot(df[x], df[y[2]], color='green')
    axes[2].set_xlabel(x_label)
    axes[2].set_ylabel(y_labels[2])
    axes[2].set_title(title + ' accZ')

    plt.show()

# Work with datasets

## squats_df

#### Create a squats_df dataframe using concat_dataframes() function

In [ ]:
columns = ['timestamp', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
squats_df_1 = pd.read_csv('data/40_Hz/original_data/train/Squats/1_HIMU-2023-08-19_13-20-06.csv', names=columns, skiprows=4)

squats_df_2 = pd.read_csv('data/40_Hz/original_data/train/Squats/2_HIMU-2023-08-19_20-18-08.csv', names=columns, skiprows=4)

squats_df_3 = pd.read_csv('data/40_Hz/original_data/train/Squats/3_HIMU-2023-08-19_20-35-09.csv', names=columns, skiprows=4)

squats_df_4 = pd.read_csv('data/40_Hz/original_data/train/Squats/4_HIMU-2023-08-20_12-13-25.csv', names=columns, skiprows=4)

squats_df_5 = pd.read_csv('data/40_Hz/original_data/train/Squats/5_HIMU-2023-08-20_12-27-08.csv', names=columns, skiprows=4)

squats_df = concat_dataframes(dfs=[squats_df_1, squats_df_2, squats_df_3, squats_df_4, squats_df_5], 
                                  timestamp_col_name='timestamp', 
                                  timecol_position=1)
squats_df.head()

In [ ]:
squats_df.info()

In [ ]:
time_diffs = squats_df['time'].diff()
measurement_time = squats_df['time'].iloc[-1] - squats_df['time'].iloc[0]
print(f"measurement_time = {measurement_time} s")
print(f"Average measurement period = {time_diffs.mean()} s")
print(f"Average measurement frequency = {1 / time_diffs.mean()} Hz")

#### Visualise accelerometer and gyroscope behaviour (Raw data) for squats_df

In [ ]:
display_three_axes(df=squats_df, x='time', y=['accX', 'accY', 'accZ'], title='Accelerometer readings (Raw Data)', 
                   x_label='Time, s', y_label='Linear acceleration, m/s^2', filename='graphs/squats_acc')
display_three_axes(df=squats_df, x='time', y=['gyrX', 'gyrY', 'gyrZ'], title='Gyroscope readings (Raw Data)', 
                   x_label='Time, s', y_label='Angular velocity, rad/s')

## leg_land_df

#### 1) Create a leg_land_df dataframe using concat_dataframes() function

In [ ]:
columns = ['timestamp', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
leg_land_df_1 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/1_HIMU-2023-08-20_15-04-38.csv', names=columns, skiprows=4)

leg_land_df_2 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/2_HIMU-2023-08-21_08-06-02.csv', names=columns, skiprows=4)

leg_land_df_3 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/3_HIMU-2023-08-21_08-17-19.csv', names=columns, skiprows=4)

leg_land_df_4 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/4_HIMU-2023-08-21_08-30-01.csv', names=columns, skiprows=4)

leg_land_df_5 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/5_HIMU-2023-08-21_08-38-01.csv', names=columns, skiprows=4)

leg_land_df = concat_dataframes(dfs=[leg_land_df_1, leg_land_df_2, leg_land_df_3, leg_land_df_4, leg_land_df_5], 
                                timestamp_col_name='timestamp', 
                                timecol_position=1)
leg_land_df.head()

In [ ]:
leg_land_df.info()

In [ ]:
time_diffs = leg_land_df['time'].diff()
measurement_time = leg_land_df['time'].iloc[-1] - leg_land_df['time'].iloc[0]
print(f"measurement_time = {measurement_time} s")
print(f"Average measurement period = {time_diffs.mean()} s")
print(f"Average measurement frequency = {1 / time_diffs.mean()} Hz")

#### 2) Visualise accelerometer and gyroscope behaviour (Raw data) for leg_land_df

In [ ]:
display_three_axes(df=leg_land_df, x='time', y=['accX', 'accY', 'accZ'], title='Accelerometer readings (Raw Data)', 
                   x_label='Time, s', y_label='Linear acceleration, m/s^2', filename='graphs/squats_acc')
display_three_axes(df=leg_land_df, x='time', y=['gyrX', 'gyrY', 'gyrZ'], title='Gyroscope readings (Raw Data)', 
                   x_label='Time, s', y_label='Angular velocity, rad/s')

#### 3) Save leg_land_df to a separate file

In [ ]:
leg_land_df.to_csv('data/40_Hz/concatenated_data/Leg_land/Leg_land_1_2023-08-21.csv', index=False)

## walk_df

#### 1) Read a walk_df from the csv-file

In [ ]:
columns = ['timestamp', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
walk_df = pd.read_csv('data/40_Hz/original_data/train/Walk/1_HIMU-2023-08-19_20-25-23.csv', names=columns, skiprows=4)

walk_df.head()

In [ ]:
walk_df.info()

##### 1.1) Add 'time' column

In [ ]:
walk_df = add_time_column(df=walk_df, timestamp_col_name='timestamp', initial_time=0, timecol_position=1)
walk_df.head()

In [ ]:
walk_df[-1:]

In [ ]:
time_diffs = walk_df['time'].diff()
measurement_time = walk_df['time'].iloc[-1] - walk_df['time'].iloc[0]
print(f"measurement_time = {measurement_time} s")
print(f"Average measurement period = {time_diffs.mean()} s")
print(f"Average measurement frequency = {1 / time_diffs.mean()} Hz")

#### 2) Visualise accelerometer and gyroscope behaviour (Raw data)

In [ ]:
display_three_axes(df=walk_df, x='time', y=['accX', 'accY', 'accZ'], title='Accelerometer readings (Raw Data)', 
                   x_label='Time, s', y_label='Linear acceleration, m/s^2')
display_three_axes(df=walk_df, x='time', y=['gyrX', 'gyrY', 'gyrZ'], title='Gyroscope readings (Raw Data)', 
                   x_label='Time, s', y_label='Angular velocity, rad/s')

#### 3) Save walk_df to a separate file

In [ ]:
walk_df.to_csv('data/40_Hz/concatenated_data/Walk/Walk_1_2023-08-21.csv', index=False)

## lateral_squat_slide_df

#### 1) Create a lateral_squat_slide_df dataframe using concat_dataframes() function

In [ ]:
columns = ['timestamp', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
lateral_squat_slide_df_1 = pd.read_csv('data/40_Hz/original_data/train/Lateral_squat_slide/1_HIMU-2023-08-21_08-10-01.csv', names=columns, skiprows=4)

lateral_squat_slide_df_2 = pd.read_csv('data/40_Hz/original_data/train/Lateral_squat_slide/2_HIMU-2023-08-21_08-23-07.csv', names=columns, skiprows=4)

lateral_squat_slide_df_3 = pd.read_csv('data/40_Hz/original_data/train/Lateral_squat_slide/3_HIMU-2023-08-21_08-56-02.csv', names=columns, skiprows=4)

lateral_squat_slide_df_4 = pd.read_csv('data/40_Hz/original_data/train/Lateral_squat_slide/4_HIMU-2023-08-21_09-20-01.csv', names=columns, skiprows=4)

lateral_squat_slide_df = concat_dataframes(dfs=[lateral_squat_slide_df_1, lateral_squat_slide_df_2, lateral_squat_slide_df_3, lateral_squat_slide_df_4], 
                                timestamp_col_name='timestamp', 
                                timecol_position=1)
lateral_squat_slide_df.head()

In [ ]:
lateral_squat_slide_df.info()

In [ ]:
time_diffs = lateral_squat_slide_df['time'].diff()
measurement_time = lateral_squat_slide_df['time'].iloc[-1] - lateral_squat_slide_df['time'].iloc[0]
print(f"measurement_time = {measurement_time} s")
print(f"Average measurement period = {time_diffs.mean()} s")
print(f"Average measurement frequency = {1 / time_diffs.mean()} Hz")

#### 2) Visualise accelerometer and gyroscope behaviour (Raw data) for lateral_squat_slide_df

In [ ]:
display_three_axes(df=lateral_squat_slide_df, x='time', y=['accX', 'accY', 'accZ'], title='Accelerometer readings (Raw Data)', 
                   x_label='Time, s', y_label='Linear acceleration, m/s^2', filename='graphs/squats_acc')
display_three_axes(df=lateral_squat_slide_df, x='time', y=['gyrX', 'gyrY', 'gyrZ'], title='Gyroscope readings (Raw Data)', 
                   x_label='Time, s', y_label='Angular velocity, rad/s')

#### 3) Save lateral_squat_slide_df to a separate file

In [ ]:
lateral_squat_slide_df.to_csv('data/40_Hz/concatenated_data/Lateral_squat_slide/Lateral_squat_slide_1_2023-08-21.csv', index=False)

## jogging_df

#### 1) Create a jogging_df dataframe using concat_dataframes() function

In [ ]:
columns = ['timestamp', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
jogging_df_1 = pd.read_csv('data/40_Hz/original_data/train/Jogging/1_HIMU-2023-08-20_09-47-23.csv', names=columns, skiprows=4)

jogging_df_2 = pd.read_csv('data/40_Hz/original_data/train/Jogging/2_HIMU-2023-08-20_10-00-50.csv', names=columns, skiprows=4)

jogging_df = concat_dataframes(dfs=[jogging_df_1, jogging_df_2], 
                                timestamp_col_name='timestamp', 
                                timecol_position=1)
jogging_df.head()

In [ ]:
jogging_df.info()

In [ ]:
time_diffs = jogging_df['time'].diff()
measurement_time = jogging_df['time'].iloc[-1] - jogging_df['time'].iloc[0]
print(f"measurement_time = {measurement_time} s")
print(f"Average measurement period = {time_diffs.mean()} s")
print(f"Average measurement frequency = {1 / time_diffs.mean()} Hz")

#### 2) Visualise accelerometer and gyroscope behaviour (Raw data) for jogging_df

In [ ]:
display_three_axes(df=jogging_df, x='time', y=['accX', 'accY', 'accZ'], title='Accelerometer readings (Raw Data)', 
                   x_label='Time, s', y_label='Linear acceleration, m/s^2', filename='graphs/squats_acc')
display_three_axes(df=jogging_df, x='time', y=['gyrX', 'gyrY', 'gyrZ'], title='Gyroscope readings (Raw Data)', 
                   x_label='Time, s', y_label='Angular velocity, rad/s')

#### 3) Save jogging_df to a separate file

In [ ]:
jogging_df.to_csv('data/40_Hz/concatenated_data/Jogging/Jogging_1_2023-08-21.csv', index=False)

## train_df

In [ ]:
columns = ['timestamp', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']

# squats
squats_df_1 = pd.read_csv('data/40_Hz/original_data/train/Squats/1_HIMU-2023-08-19_13-20-06.csv', names=columns, skiprows=4)
squats_df_2 = pd.read_csv('data/40_Hz/original_data/train/Squats/2_HIMU-2023-08-19_20-18-08.csv', names=columns, skiprows=4)
squats_df_3 = pd.read_csv('data/40_Hz/original_data/train/Squats/3_HIMU-2023-08-19_20-35-09.csv', names=columns, skiprows=4)
squats_df_4 = pd.read_csv('data/40_Hz/original_data/train/Squats/4_HIMU-2023-08-20_12-13-25.csv', names=columns, skiprows=4)
squats_df_5 = pd.read_csv('data/40_Hz/original_data/train/Squats/5_HIMU-2023-08-20_12-27-08.csv', names=columns, skiprows=4)

# leg_land
leg_land_df_1 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/1_HIMU-2023-08-20_15-04-38.csv', names=columns, skiprows=4)
leg_land_df_2 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/2_HIMU-2023-08-21_08-06-02.csv', names=columns, skiprows=4)
leg_land_df_3 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/3_HIMU-2023-08-21_08-17-19.csv', names=columns, skiprows=4)
leg_land_df_4 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/4_HIMU-2023-08-21_08-30-01.csv', names=columns, skiprows=4)
leg_land_df_5 = pd.read_csv('data/40_Hz/original_data/train/Leg_land/5_HIMU-2023-08-21_08-38-01.csv', names=columns, skiprows=4)

# walk
walk_df = pd.read_csv('data/40_Hz/original_data/train/Walk/1_HIMU-2023-08-19_20-25-23.csv', names=columns, skiprows=4)

# lateral squat slide
lateral_squat_slide_df_1 = pd.read_csv('data/40_Hz/original_data/train/Lateral_squat_slide/1_HIMU-2023-08-21_08-10-01.csv', names=columns, skiprows=4)
lateral_squat_slide_df_2 = pd.read_csv('data/40_Hz/original_data/train/Lateral_squat_slide/2_HIMU-2023-08-21_08-23-07.csv', names=columns, skiprows=4)
lateral_squat_slide_df_3 = pd.read_csv('data/40_Hz/original_data/train/Lateral_squat_slide/3_HIMU-2023-08-21_08-56-02.csv', names=columns, skiprows=4)
lateral_squat_slide_df_4 = pd.read_csv('data/40_Hz/original_data/train/Lateral_squat_slide/4_HIMU-2023-08-21_09-20-01.csv', names=columns, skiprows=4)

# jogging
jogging_df_1 = pd.read_csv('data/40_Hz/original_data/train/Jogging/1_HIMU-2023-08-20_09-47-23.csv', names=columns, skiprows=4)
jogging_df_2 = pd.read_csv('data/40_Hz/original_data/train/Jogging/2_HIMU-2023-08-20_10-00-50.csv', names=columns, skiprows=4)

# concatenate all the small dataframes
train_df = concat_dataframes(dfs=[squats_df_1, squats_df_2, squats_df_3, squats_df_4, squats_df_5,
                                  leg_land_df_1, leg_land_df_2, leg_land_df_3, leg_land_df_4, leg_land_df_5,
                                  walk_df,
                                  lateral_squat_slide_df_1, lateral_squat_slide_df_2, lateral_squat_slide_df_3, lateral_squat_slide_df_4,
                                  jogging_df_1, jogging_df_2], 
                             timestamp_col_name='timestamp', 
                             timecol_position=1)

train_df.head()

In [ ]:
train_df.info()

In [ ]:
time_diffs = train_df['time'].diff()
measurement_time = train_df['time'].iloc[-1] - train_df['time'].iloc[0]
print(f"measurement_time = {measurement_time} s")
print(f"Average measurement period = {time_diffs.mean()} s")
print(f"Average measurement frequency = {1 / time_diffs.mean()} Hz")

#### 2) Visualise accelerometer and gyroscope behaviour (Raw data) for train_df

In [ ]:
display_three_axes(df=train_df, x='time', y=['accX', 'accY', 'accZ'], title='Accelerometer readings (Raw Data)', 
                   x_label='Time, s', y_label='Linear acceleration, m/s^2', filename='graphs/squats_acc')
display_three_axes(df=train_df, x='time', y=['gyrX', 'gyrY', 'gyrZ'], title='Gyroscope readings (Raw Data)', 
                   x_label='Time, s', y_label='Angular velocity, rad/s')

#### 3) Save train_df to a separate file

In [ ]:
#train_df.drop('timestamp', axis=1, inplace=True)
train_df.to_csv('data/40_Hz/concatenated_data/All/Train_1_2023-08-21.csv', index=False)

In [ ]:
read_train_df = pd.read_csv('data/40_Hz/concatenated_data/All/Train_1_2023-08-21.csv')
read_train_df.head()

## test_df

In [ ]:
columns = ['timestamp', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
test_df = pd.read_csv('data/40_Hz/original_data/test/Test1_HIMU-2023-08-20_09-31-52.csv', names=columns, skiprows=4)

test_df.head()

In [ ]:
test_df.info()

#### 1) Add 'time' column

In [ ]:
test_df = add_time_column(df=test_df, timestamp_col_name='timestamp', initial_time=0, timecol_position=2)
test_df.head()

In [ ]:
test_df[-1:]

In [ ]:
time_diffs = test_df['time'].diff()
measurement_time = test_df['time'].iloc[-1] - test_df['time'].iloc[0]
print(f"measurement_time = {measurement_time} s")
print(f"Average measurement period = {time_diffs.mean()} s")
print(f"Average measurement frequency = {1 / time_diffs.mean()} Hz")

#### 2) Visualise accelerometer and gyroscope behaviour (Raw data)

In [ ]:
display_three_axes(df=test_df, x='time', y=['accX', 'accY', 'accZ'], title='Accelerometer readings (Raw Data)', 
                   x_label='Time, s', y_label='Linear acceleration, m/s^2')
display_three_axes(df=test_df, x='time', y=['gyrX', 'gyrY', 'gyrZ'], title='Gyroscope readings (Raw Data)', 
                   x_label='Time, s', y_label='Angular velocity, rad/s')

## activities_df

In [34]:
labels_df = create_activities_df(path='data/40_Hz/labeled_data/Train_1_labeled_1.csv')
labels_df

,start_time,end_time,activity
0,0.000,4.774,No activity
1,4.774,80.000,Squat
2,80.000,87.279,No activity
3,87.279,158.028,Squat
4,158.028,162.004,No activity
5,162.004,233.884,Squat
6,233.884,241.239,No activity
7,241.239,310.060,Squat
8,310.060,316.947,No activity
9,316.947,386.844,Squat


In [35]:
df = pd.read_csv('data/40_Hz/concatenated_data/All/Train_1_2023-08-21.csv')
df.head()

,timestamp,time,accX,accY,accZ,gyrX,gyrY,gyrZ
0,1692440406933,0.000,0.14895,-0.06000,9.775050,0.106150,-0.018975,-1.117325
1,1692440406976,0.043,-0.05400,0.04200,9.748950,0.110550,-0.163075,-1.241488
2,1692440407003,0.070,0.23295,-0.12405,9.739051,0.096937,-0.311437,-1.159812
3,1692440407037,0.104,-0.18495,-0.01200,9.691051,0.079475,-0.309375,-0.900900
4,1692440407057,0.124,-0.46305,0.13605,10.146001,0.065037,-0.293013,-0.771787


In [36]:
add_activity_column(path='data/40_Hz/labeled_data/Train_1_labeled_1.csv', df=df, df_merge_col='time')
df.head()

,timestamp,time,accX,accY,accZ,gyrX,gyrY,gyrZ,activity
0,1692440406933,0.000,0.14895,-0.06000,9.775050,0.106150,-0.018975,-1.117325,No activity
1,1692440406976,0.043,-0.05400,0.04200,9.748950,0.110550,-0.163075,-1.241488,No activity
2,1692440407003,0.070,0.23295,-0.12405,9.739051,0.096937,-0.311437,-1.159812,No activity
3,1692440407037,0.104,-0.18495,-0.01200,9.691051,0.079475,-0.309375,-0.900900,No activity
4,1692440407057,0.124,-0.46305,0.13605,10.146001,0.065037,-0.293013,-0.771787,No activity


### Check that the 'activity' column is filled in correctly

In [37]:
df[df['time'] > 4.730][:5]

,timestamp,time,accX,accY,accZ,gyrX,gyrY,gyrZ,activity
163,1692440411679,4.746,0.07500,-0.23895,9.934951,0.008250,-0.004262,0.039462,No activity
164,1692440411707,4.774,0.23505,-0.31605,9.904051,0.025163,-0.023512,0.005775,Squat
165,1692440411742,4.809,0.35205,-0.33705,9.555000,0.024750,-0.048262,-0.113300,Squat
166,1692440411776,4.843,0.13905,-0.28695,9.495001,0.015950,-0.065312,-0.138325,Squat
167,1692440411806,4.873,0.05295,-0.21795,9.571051,-0.002200,-0.095425,-0.111925,Squat


In [38]:
df[df['time'] > 79.900][:5]

,timestamp,time,accX,accY,accZ,gyrX,gyrY,gyrZ,activity
2699,1692440486846,79.913,0.32295,0.69105,8.236051,0.314462,0.136675,0.021725,Squat
2700,1692440486881,79.948,-0.09705,0.24300,9.897000,0.620950,-0.123887,-0.787737,Squat
2701,1692440486904,79.971,0.13905,0.05205,10.000051,0.723525,-0.213950,-1.060950,Squat
2702,1692440486933,80.000,-0.33600,-0.15000,10.744051,0.751162,-0.268263,-1.146062,No activity
2703,1692440486962,80.029,0.05505,-0.17205,11.302051,0.591800,-0.264275,-0.802863,No activity


In [39]:
df[df['time'] > 87.200][:5]

,timestamp,time,accX,accY,accZ,gyrX,gyrY,gyrZ,activity
2948,1692465490666,87.223,-0.45105,0.04005,9.468000,0.007700,0.050875,0.235950,No activity
2949,1692465490693,87.250,-0.29505,0.05700,9.685950,0.040975,0.077137,0.023787,No activity
2950,1692465490722,87.279,-0.38895,-0.02400,9.765000,0.065725,0.086763,-0.108213,Squat
2951,1692465490748,87.305,-0.32895,-0.05205,9.555000,0.094738,0.061325,-0.355025,Squat
2952,1692465490778,87.335,-0.19695,-0.01905,9.568951,0.092262,0.037262,-0.354750,Squat


In [40]:
df[df['time'] > 627.850][:5]

,timestamp,time,accX,accY,accZ,gyrX,gyrY,gyrZ,activity
21475,1692595803977,627.864,-0.15195,-0.03105,9.523951,0.041800,0.049362,-0.232513,No activity
21476,1692595804007,627.894,-0.25995,-0.03795,9.721951,0.025850,0.043037,-0.212575,Leg land
21477,1692595804030,627.917,-0.37395,0.09495,9.625951,0.003987,0.031625,-0.178888,Leg land
21478,1692595804061,627.948,0.05805,0.24600,10.062000,-0.040700,0.009625,-0.142725,Leg land
21479,1692595804088,627.975,0.20595,0.81795,9.181050,-0.040287,0.001650,-0.125950,Leg land


In [41]:
df[df['time'] > 798.500][:5]

,timestamp,time,accX,accY,accZ,gyrX,gyrY,gyrZ,activity
27352,1692465926330,798.504,-0.04395,-0.00195,10.684051,0.009212,-0.013062,-0.374550,No activity
27353,1692465926358,798.532,0.07005,-0.30405,10.324950,-0.005088,0.032037,-0.372350,No activity
27354,1692465926386,798.560,0.03600,-0.40200,10.900050,-0.025300,0.020075,-0.337838,Walk
27355,1692465926414,798.588,0.04605,0.17205,11.223001,-0.086212,-0.026812,-0.216562,Walk
27356,1692465926441,798.615,-0.19995,0.66495,9.873000,-0.115913,-0.041938,-0.152487,Walk


In [42]:
df[df['time'] > 1667.950][:5]

,timestamp,time,accX,accY,accZ,gyrX,gyrY,gyrZ,activity
57160,1692514852942,1667.968,-0.23595,-0.30795,3.83400,-0.096937,-0.060362,0.114950,No activity
57161,1692514852969,1667.995,0.20100,-0.52200,1.96905,-0.075625,-0.035612,0.206113,Jogging
57162,1692514852998,1668.024,0.28800,-1.25595,-1.17495,-0.104637,0.080162,0.643637,Jogging
57163,1692514853022,1668.048,-0.71700,-1.38795,-1.42395,-0.170088,0.108350,0.944075,Jogging
57164,1692514853052,1668.078,-2.34000,-2.59200,6.41700,-0.270600,0.090612,1.204638,Jogging


In [43]:
df[df['time'] > 1847.925][:5]

,timestamp,time,accX,accY,accZ,gyrX,gyrY,gyrZ,activity
63378,1692515032902,1847.928,0.76905,0.08100,11.812051,0.250663,-0.307175,0.918500,Jogging
63379,1692515032935,1847.961,-1.14300,2.49105,6.474000,0.109863,-0.299887,0.498163,Jogging
63380,1692515032964,1847.990,-2.54805,0.31305,10.692000,0.088550,-0.376612,0.366438,Jogging
63381,1692515032992,1848.018,0.49095,-4.41105,15.985950,0.175038,-0.376063,0.364650,No activity
63382,1692515033022,1848.048,0.86895,-4.54800,15.922050,0.285863,-0.246400,0.338662,No activity
